# Retroactive DCF Calculation
Using actual cash flow data, rather than predicted cash flow based on growth assumptions, this notebook retroactively computes a DCF valuation for a company at the time a deal was proposed. By comparing this valuation to the terms of the proposed acquisition deal, one can determine whether the terms of the deal proved advantageous for the target or acquirer independent of overall market performance. This comarison can be used to label the acqusition dataset. 

In [328]:
import lseg.data as ld
from lseg.data.content import search
from datetime import date, timedelta
import numpy as np
import pandas as pd


## Read the Deals Dataset

In [329]:
deals_df = pd.read_csv('incomplete_acquisition_deals.csv')
deals_df.head()

,AcquirerName,TargetName,TransactionAnnouncementDate,PercentageOfSharesAcquired,AcquirerRegion,TargetRegion,PricePerShare,CurrencyCodeOfTransaction,DealPurpose,FormOfTransactionName,...,TargetMidIndustry,AnalystEstimatedValue,RatioOfTransactionValueToNetIncome,RatioOfTransactionValueToCashFlow,RatioOfTransactionValueToSales,RatioOfRankvalueToNetIncome,RatioOfRankvalueToCashFlow,RatioOfRankvalueToSales,TransactionValueIncludingNetDebtOfTarget,TransactionEffectiveDate
0,['Comcast Corp'],['TFCF Corp'],2018-06-13,0,Americas,Americas,35.000,US,"['Sell a loss making/bankrupt operation', 'Str...",Merger,...,Cable,NaN,21.308,10.182,2.220,25.457,12.165,2.652,77464.372,2018-06-13
1,['Chevron Corp'],['Anadarko Petroleum Corp'],2019-04-12,0,Americas,Americas,64.996,US,"['Create synergies, eliminate duplicate servic...",Merger,...,Oil & Gas,NaN,62.177,6.162,3.631,63.345,6.277,3.699,48712.296,2019-04-12
2,['Hong Kong Exchanges and Clearing Ltd'],['London Stock Exchange Group PLC'],2019-09-11,0,Asia Pacific Excluding Central Asia,Europe,103.287,STG,"['Expand presence in new/foreign markets', 'Cr...",Merger,...,Brokerage,NaN,110.412,51.970,25.666,110.412,51.970,25.666,37259.153,2019-09-11
3,['Xerox Holdings Corp'],['HP Inc'],2019-11-06,0,Americas,Americas,23.938,US,['Strengthen existing operations/expand presen...,Merger,...,Computers & Peripherals,33000.0,11.742,6.804,0.594,11.961,6.930,0.606,35524.904,2019-11-06
4,['Alimentation Couche-Tard Inc'],['Carrefour SA'],2021-01-12,0,Americas,Europe,24.414,EUR,"['Expand presence in new/foreign markets', 'St...",Merger,...,FOOD/BEVERAGES,NaN,18.949,3.631,0.224,32.799,6.284,0.388,34151.922,2021-01-12


# Functions to Calculate the Retroactive DCF
$\text{DCF} = \sum_{i=1}^{n} \frac{\text{CF}_i}{(1 + r)^i} + \frac{\text{TV}}{(1 + r)^n}\\$ 
These functions use the actual cash flows gather via the LSEG API rather than projected cash flows. The terminal value is implemented once there no longer is any actual cash flow data and computed with the Exit Mutliple Method Perpetuity Growth Model. A weighted averaging approach is used to estimate g (the growth rate). 

In [330]:
# Compute the Projected Growth Rate for the Perpetuity Growth Rate using a Weighted Average Approach to include momentum
def estimate_g(n, actual_cash_flows):
    # Calculate year-on-year growth rates
    growth_rates = []
    for i in range(1, len(actual_cash_flows)):
        growth_rate = (actual_cash_flows[i] - actual_cash_flows[i - 1]) / actual_cash_flows[i - 1]
        growth_rates.append(growth_rate)
    
    # Assign weights (more recent years get higher weights)
    weights = np.arange(n, 0, -1)  # e.g., if n=4, weights = [4, 3, 2, 1]
    
    # Calculate the weighted average growth rate
    numerator = sum(rate * weight for rate, weight in zip(growth_rates, weights[:len(growth_rates)]))
    denominator = sum(weights[:len(growth_rates)])
    
    g = numerator / denominator if denominator != 0 else 0  # Avoid division by zero
    return g

# Uses Exit Mutliple Method to compute terminal value
def compute_terminal_value(ebitda_last_year, exit_multiple):
    terminal_value = ebitda_last_year * exit_multiple
    return terminal_value

def retroactive_dcf(discount_rate, n, actual_cash_flows, ebitda_last_year, exit_mutliple):
    # Estimate the growth rate
    growth_rate = estimate_g(n, actual_cash_flows)
    print(growth_rate)

    # Calculate the terminal value based on the last actual cash flow
    terminal_value = compute_terminal_value(ebitda_last_year, exit_mutliple)

    # Discount the cash flows and terminal value back to present value
    dcf_value = sum(cf / (1 + discount_rate) ** (i + 1) for i, cf in enumerate(actual_cash_flows))
    dcf_value += terminal_value / (1 + discount_rate) ** n

    return dcf_value


In [331]:
import lseg.data as ld
from datetime import date, timedelta

ld.open_session()

# Create date objects for today and 10 years ago
today = date.today()
five_years_ago = today - timedelta(days=365*5)

df = ld.get_history(
    universe="IBM",
    fields=["TR.F.CF"],
    interval="yearly",
    start=five_years_ago,
    end=today
)

df.head()


# fields=["TR.F.CF", "TR.CFPSMean", "TR.CFPSMedian", "TR.F.NetCashFlowOp", "TR.F.NetCashFlowInvst", "TR.NetCashFlowFin", "TR.MnATargetSharesOutstandingMRQ", "TR.CompanyMarketCap"],

IBM,Cash Flow
Date,
2018-12-31,15240000000
2019-12-31,13351000000
2020-12-31,10737000000
2021-12-31,11041000000
2022-12-31,6655000000


## Functions to Obtain Yearly Cash Flow Data, EBITDA closest to the time of the deal, and EBITDA Exit Multiplier From the LSEG API

In [332]:
def obtain_cash_flow_data(company_ticker, deal_date, end_date=date.today()):

    ld.open_session() # Must be running LSEG Desktop App

    df = ld.get_history(
        universe=company_ticker,
        fields=["TR.F.CF"],
        interval="yearly",
        start=deal_date,
        end=end_date
    )

    ld.close_session()

    return df

def obtain_most_recent_ebitda(company_ticker, deal_date):

    ld.open_session()

    # date object for one year prior to deal
    one_year_before_deal = deal_date - timedelta(days=365)

    df = ld.get_history(
        universe=company_ticker,
        fields=["TR.EBITDAActValue"],
        interval="monthly",
        start=one_year_before_deal,
        end=deal_date
    )

    print(df)
    most_recent_ebitda = df.iloc[-1, 0]

    ld.close_session()

    return most_recent_ebitda

def obtain_ebitda_exit_mutliple(common_name):
    
    sector_ebitda_exit_multiples = {
    "Energy": 5.5,
    "Materials": 6.0,
    "Industrials": 7.0,
    "Consumer Discretionary": 8.5,
    "Consumer Staples": 10.5,
    "Health Care": 12.0,
    "Financials": 8.0,
    "Information Technology": 12.5,
    "Communication Services": 8.0,
    "Utilities": 7.5,
    "Real Estate": 16.0
    }

    ld.open_session()

    try:
        gics_df = ld.discovery.search(
            view=ld.discovery.Views.ORGANISATIONS,
            select="GicsName",
            filter=f"CommonName eq '{common_name}'",
            top=10000
        )

        gics_sector = gics_df.iloc[0, 0][0]  # Adjust indexing if needed
    except Exception as e:
        print(f"Error occurred: {e}")  # Optional: log the error for debugging
        return 8.0  # Return default value if search fails
    finally:
        ld.close_session()

    return sector_ebitda_exit_multiples.get(gics_sector, 8.0)  # Return default if sector not found


## Test Case: Retroactive DCF for Apple (AAPL) with cash flow data obtained via the LSEG API

In [333]:
ticker = "AAPL.O"
deal_date = date.today() - timedelta(days=365*10)

cash_flow_df = obtain_cash_flow_data(ticker, deal_date)
cash_flows = cash_flow_df['Cash Flow'].values
print(cash_flows)

recent_ebitda = obtain_most_recent_ebitda(ticker, deal_date)
print(recent_ebitda)

# exit_mult = obtain_ebitda_exit_mutliple(ticker, deal_date)
# print(exit_mult)

valuation = retroactive_dcf(0.1, 10, cash_flows, recent_ebitda, 12.5)

print(f"The Rectroactive DCF Valuation for Apple Inc 10 Years Ago is: ${valuation}")

<IntegerArray>
[ 47510000000,  64651000000,  56192000000,  58508000000,  71934000000,
  67803000000,  68467000000, 105964000000, 110907000000, 108514000000]
Length: 10, dtype: Int64
AAPL.O               EBITDA - Actual
Date                                
2013-10-28 16:30:00      58009000000
2014-10-20 16:30:00      60449000000
60449000000
0.11774428413889908
The Rectroactive DCF Valuation for Apple Inc 10 Years Ago is: $726500453245.9514


## Find the company ticker so we can execute a get_history() search

In [334]:
def find_ticker(company_name, deal_date):
    
    ld.close_session()
    ld.open_session()

    print(f"Company Name: {company_name}")

    try:
        ticker_df = ld.discovery.search(
            view=ld.discovery.Views.ORGANISATIONS,
            select="PrimaryRICTickerSymbol", 
            filter=f"CommonName eq '{company_name}'",
            top=10000
        )
    except Exception as e:
        return "NO_TICKER_FOUND"
    
    if ticker_df.empty:
        return "NO_TICKER_FOUND"

    partial_ticker = ticker_df.iloc[0,0] # Ticker but doesn't contain a suffix which is often necessary for a Python Data Library get_history() search
    print(f"PARTIAL_TICKER: {partial_ticker}")

    ordinary_share_suffixes = [
        "",       # No suffix (common for NYSE, NASDAQ, most U.S. stocks)
        ".O",     # NASDAQ (e.g., AAPL.O for Apple Inc.)
        ".N",     # NYSE (e.g., IBM.N for IBM)
        ".L",     # London Stock Exchange (LSE) (e.g., BP.L for BP)
        ".T",     # Tokyo Stock Exchange (TSE) (e.g., 7203.T for Toyota)
        ".HK",    # Hong Kong Stock Exchange (HKEX) (e.g., 0005.HK for HSBC)
        ".AX",    # Australian Securities Exchange (ASX) (e.g., CBA.AX for Commonwealth Bank)
        ".TO",    # Toronto Stock Exchange (TSX) (e.g., RY.TO for Royal Bank of Canada)
        ".PA",    # Euronext Paris (e.g., AIR.PA for Airbus)
        ".F",     # Frankfurt Stock Exchange (FSE) (e.g., BMW.F for BMW)
        ".SI",    # Singapore Exchange (SGX) (e.g., D05.SI for DBS Group)
        ".SW",    # SIX Swiss Exchange (e.g., NESN.SW for Nestlé)
        ".DE",    # Xetra (Deutsche Börse) (e.g., VOW3.DE for Volkswagen)
        ".MI",    # Borsa Italiana (Milan Stock Exchange) (e.g., ENI.MI for Eni S.p.A.)
        ".KS",    # Korea Exchange (KRX) (e.g., 005930.KS for Samsung)
        ".KQ",    # KOSDAQ (Korean secondary market) (e.g., 035720.KQ for Kakao)
        ".SA",    # B3 - Brazil Stock Exchange (B3 S.A.) (e.g., PETR4.SA for Petrobras)
        ".IS",    # Borsa Istanbul (e.g., THYAO.IS for Turkish Airlines)
        ".OL",    # Oslo Stock Exchange (e.g., EQNR.OL for Equinor)
        ".MC",    # Madrid Stock Exchange (BME) (e.g., SAN.MC for Santander)
        ".ST",    # Stockholm Stock Exchange (OMX Stockholm) (e.g., VOLVb.ST for Volvo)
        ".AS",    # Euronext Amsterdam (e.g., RDSA.AS for Royal Dutch Shell)
        ".NZ",    # New Zealand Exchange (NZX) (e.g., FPH.NZ for Fisher & Paykel)
        ".BR",    # Euronext Brussels (e.g., ABI.BR for Anheuser-Busch InBev)
        ".AT",    # Vienna Stock Exchange (e.g., OMV.AT for OMV Group)
        ".VN",    # Ho Chi Minh Stock Exchange (HOSE) (e.g., VIC.VN for Vingroup)
        ".J",     # Johannesburg Stock Exchange (JSE) (e.g., MTN.J for MTN Group)
        ".JK",    # Indonesia Stock Exchange (IDX) (e.g., BBCA.JK for Bank Central Asia)
        ".TW",    # Taiwan Stock Exchange (TWSE) (e.g., 2330.TW for TSMC)
        ".Z",     # Zurich (Swiss Market) (e.g., NESN.Z for Nestlé on Swiss Market)
    ]

    # Now, we try to find the appropriate suffix for the partial ticker
    for suffix in ordinary_share_suffixes:
        
        possible_ticker = partial_ticker + suffix

        try:
            # Attempt to obtain cash flow data
            cash_flow_data = obtain_cash_flow_data(possible_ticker, deal_date)
            if not isinstance(cash_flow_data, str):  # Check if it's not an error message
                return possible_ticker  # Return the valid ticker if data retrieval succeeds
        except Exception as e:
            # Handle exceptions that may arise during cash flow data retrieval
            continue  # Skip to the next suffix if it fails

    return "NO_TICKER_FOUND"  # If no valid ticker is found

# Test
deal_date = date.today() - timedelta(days=365*10)
print(find_ticker("Newmont Corporation", deal_date))

Company Name: Newmont Corporation
PARTIAL_TICKER: NEM
NEM


## Gather Cash Flow Data Using the LSEG API and then Compute the Retroactive DCF
We create a new dataframe where the search process does not fail

In [335]:
# Add new columns for Ticker and RetroactiveDCFValuation
deals_df['Ticker'] = None
deals_df['RetroactiveDCFValuation'] = None

# Define the discount rate 
discount_rate = 0.1  # 10%

# Iterate through the DataFrame row by row
rows_to_drop = []  # Store indices of rows to drop

for index, row in deals_df.iterrows():
    company_name = row['TargetName'].strip("[]'\"")
    transaction_date = date.fromisoformat(row['TransactionAnnouncementDate'])

    print(f"Processing {company_name} with transaction date {transaction_date}")

    # Find the ticker using the find_ticker function
    ticker = find_ticker(company_name, transaction_date)

    if ticker == "NO_TICKER_FOUND":
        print(f"Could not find ticker for {company_name}. Marking row for deletion.")
        rows_to_drop.append(index)  # Mark this row for deletion
        continue  # Skip to the next iteration

    deals_df.at[index, 'Ticker'] = ticker  # Add the ticker to the DataFrame
    print(f"Found ticker: {ticker} for {company_name}")

    try:
        # Obtain cash flow data
        cash_flow_data = obtain_cash_flow_data(ticker, transaction_date)

        # Check if cash flow data is not empty
        if cash_flow_data.empty:
            print(f"No cash flow data found for ticker {ticker}. Marking row for deletion.")
            rows_to_drop.append(index)  # Mark this row for deletion
            continue  # Skip to the next iteration

        # Get actual cash flows as a list
        actual_cash_flows = cash_flow_data['Cash Flow'].tolist()
        print(f"Retrieved cash flow data for {ticker}: {actual_cash_flows}")

        # Obtain the most recent EBITDA
        most_recent_ebitda = obtain_most_recent_ebitda(ticker, transaction_date)
        print(f"Most recent EBITDA for {ticker}: {most_recent_ebitda}")

        # Obtain the exit multiple 
        exit_multiple = obtain_ebitda_exit_mutliple(company_name)
        print(f"Exit multiple for {ticker}: {exit_multiple}")

        # Compute the retroactive DCF valuation
        n = len(actual_cash_flows)  # Number of years of cash flows
        retroactive_valuation = retroactive_dcf(discount_rate, n, actual_cash_flows, most_recent_ebitda, exit_multiple)
        print(f"Computed retroactive DCF valuation for {ticker}: {retroactive_valuation}")

        # Add the computed valuation to the DataFrame
        deals_df.at[index, 'RetroactiveDCFValuation'] = retroactive_valuation

    except Exception as e:
        print(f"Error processing {company_name}: {e}")  # Log the error for debugging
        rows_to_drop.append(index)  # Mark this row for deletion

# Drop rows marked for deletion
deals_df.drop(index=rows_to_drop, inplace=True)

# Display the updated DataFrame
print("Updated deals DataFrame:")
print(deals_df.head())


Processing TFCF Corp with transaction date 2018-06-13
Company Name: TFCF Corp
Could not find ticker for TFCF Corp. Marking row for deletion.
Processing Anadarko Petroleum Corp with transaction date 2019-04-12
Company Name: Anadarko Petroleum Corp
Could not find ticker for Anadarko Petroleum Corp. Marking row for deletion.
Processing London Stock Exchange Group PLC with transaction date 2019-09-11
Company Name: London Stock Exchange Group PLC
PARTIAL_TICKER: LSEG
Found ticker: LSEG.L for London Stock Exchange Group PLC
Retrieved cash flow data for LSEG.L: [847000000, 834000000, 727000000, 2127000000, 2849000000]
LSEG.L               EBITDA - Actual
Date                                
2018-03-02 07:01:00        915000000
2019-03-01 07:02:00       1066000000
Most recent EBITDA for LSEG.L: 1066000000
Exit multiple for LSEG.L: 8.0
0.41900900033116795
Computed retroactive DCF valuation for LSEG.L: 10522453570.608065
Processing HP Inc with transaction date 2019-11-06
Company Name: HP Inc
PAR

An error occurred while requesting URL('http://localhost:9010/api/udf').
	ReadTimeout('timed out')


Could not find ticker for Zhejiang Fuchunjiang Environmental Thermoelectric Co Ltd. Marking row for deletion.
Processing Shangying Global Co Ltd with transaction date 2020-03-10
Company Name: Shangying Global Co Ltd
Could not find ticker for Shangying Global Co Ltd. Marking row for deletion.
Processing Heilongjiang Interchina Water Treatment Co Ltd with transaction date 2021-05-27
Company Name: Heilongjiang Interchina Water Treatment Co Ltd
Could not find ticker for Heilongjiang Interchina Water Treatment Co Ltd. Marking row for deletion.
Processing Misho Ecology & Landscape Co Ltd with transaction date 2021-01-07
Company Name: Misho Ecology & Landscape Co Ltd
Could not find ticker for Misho Ecology & Landscape Co Ltd. Marking row for deletion.
Processing Xingyuan Environment Technology Co Ltd with transaction date 2020-11-17
Company Name: Xingyuan Environment Technology Co Ltd
Could not find ticker for Xingyuan Environment Technology Co Ltd. Marking row for deletion.
Processing Huttig

In [337]:
deals_df.to_csv('updated_acquisition_deals.csv', index=False)

ImportError: cannot import name 'SequenceNotStr' from 'pandas._typing' (/opt/anaconda3/lib/python3.11/site-packages/pandas/_typing.py)

In [338]:
import csv

# Open a file in write mode
with open('updated_acquisition_deals.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(deals_df.columns)
    
    # Write the data rows
    writer.writerows(deals_df.values)

In [ ]:
ld.open_session()

gics_df = ld.discovery.search(
            view=ld.discovery.Views.ORGANISATIONS,
            select="GicsName",
            filter=f"CommonName eq 'Entain PLC'",
            top=10000
        )

print(gics_df.iloc[0,0][0])

ld.close_session()

Consumer Discretionary
